In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (12, 3)
plt.rcParams["axes.grid"] = True

In [ ]:
def describe_df(df):
    stats = df.copy()
    stats['Total'] = stats.sum(axis=1)
    return pd.DataFrame(stats.describe()).iloc[[3,1,-1,2]]



In [ ]:
df_prep = pd.read_csv("../csv/timings_pre_processing.csv", delimiter=';')
df_stack = pd.read_csv("../csv/timings_stacking.csv", delimiter=';')
df_post = pd.read_csv("../csv/timings_post_process.csv", delimiter=';')
df_sd = pd.read_csv("../csv/stacking_data.csv", delimiter=';')

# Timings

### Pre-processing timings

In [ ]:
describe_df(df_prep)

In [ ]:
df_prep.plot(xlabel="frame", ylabel="exection time (ms)", title="pre-process execution times")
plt.show()

### Stacking timings

In [ ]:
describe_df(df_stack)

In [ ]:
df_stack.plot(xlabel="frame", ylabel="exection time (ms)", title="stacker execution times")
plt.show()

### Post-processing timings

In [ ]:
describe_df(df_post)

In [ ]:
df_post.plot(xlabel="frame", ylabel="exection time (ms)", title="post-process execution times")
plt.show()

# Parameters

### Alignment parameters

In [ ]:
describe_df(df_sd)

In [ ]:
df_sd[['x_trans', 'y_trans']].plot(
    
    xlabel="frame", 
    ylabel="translation (px)", 
    title="applied translation for each frame")

plt.show()

In [ ]:
import math
(df_sd['rotation']/(2*math.pi)*360).plot(
    
    xlabel="frame", 
    ylabel="image rotation (degrees)",
    title="applied rotation for each frame")
plt.show()

In [ ]:
(df_sd['ratio']*100).plot(
    
    kind="bar",
    xlabel="frame",
    ylabel="detection surface ratio (%)",
    title="match detection surface ratio",
    grid=False
)
plt.show()

In [ ]:
df_sd[['matches', 'req_matches']].plot(
    
    xlabel="frame",
    ylabel="match count",
#     ylim=(0, df_sd['matches'].max() + 10),
    title="frame vs reference match count vs required minimum"
)
plt.show()

In [ ]:
df_sd['accepted'].transform(lambda x: 1 if x else 0).plot(
    
    kind="bar",
    grid=False,
    xlabel="frame",
    ylabel="Accepted",
    title="image compliance to minimum match count requirements. 1: Accepted, 0: Rejected"
    
)
plt.show()

In [ ]:
df_sd['accepted'].transform(lambda x: "accepted" if x else "rejected").value_counts().plot(
    
    kind="pie",
    autopct='%1.1f%%',
    title="accepted/rejected frames distribution",
    ylabel=""
)
plt.show()